<a href="https://colab.research.google.com/github/Abhijitgayen/SNA_As_Msc_projects/blob/main/10_Word2Vec_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/minsuk-heo/tf2/blob/master/jupyter_notebooks/10.Word2Vec_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentence Classification using LSTM and Pretrained Word2Vec

We will train and test sentence classification using LSTM, and Pretrained Word2Vec.  
You can find visualization of our code below.

##Add libary

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  pass

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd

import tensorflow_hub as hub
import numpy as np

In [ ]:
# Load Pretrained Word2Vec
embed = hub.load("https://tfhub.dev/google/Wiki-words-250/2")

In [ ]:
def get_max_length(df):
    """
    get max token counts from train data, 
    so we use this number as fixed length input to RNN cell
    """
    max_length = 0
    for row in df['sentiment']:
        if len(row.split(" ")) > max_length:
            max_length = len(row.split(" "))
    return max_length

def get_word2vec_enc(reviews):
    """
    get word2vec value for each word in sentence.
    concatenate word in numpy array, so we can use it as RNN input
    """
    encoded_reviews = []
    for review in reviews:
        tokens = review.split(" ")
        word2vec_embedding = embed(tokens)
        encoded_reviews.append(word2vec_embedding)
    return encoded_reviews
        
def get_padded_encoded_reviews(encoded_reviews):
    """
    for short sentences, we prepend zero padding so all input to RNN has same length
    """
    padded_reviews_encoding = []
    for enc_review in encoded_reviews:
        zero_padding_cnt = max_length - enc_review.shape[0]
        pad = np.zeros((1, 250))
        for i in range(zero_padding_cnt):
            enc_review = np.concatenate((pad, enc_review), axis=0)
        padded_reviews_encoding.append(enc_review)
    return padded_reviews_encoding

def sentiment_encode(sentiment):
    """
    return one hot encoding for Y value
    """
    if sentiment == 0:
        return [1,0]
    if sentiment == 2:
        return [0.5,0.5]
    else:
        return [0,1]
    
def preprocess(df):
    """
    encode text value to numeric value
    """
    # encode words into word2vec
    reviews = df['sentiment'].tolist()
    
    encoded_reviews = get_word2vec_enc(reviews)
    padded_encoded_reviews = get_padded_encoded_reviews(encoded_reviews)
    # encoded sentiment
    sentiments = df['lebal'].tolist()
    encoded_sentiment = [sentiment_encode(sentiment) for sentiment in sentiments]
    X = np.array(padded_encoded_reviews)
    Y = np.array(encoded_sentiment)
    return X, Y 

# Preprocess (encode text to number)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
tweetsDatas=pd.read_csv('drive/MyDrive/data_set/sts-test_data_set/training.csv', names=['lebal','id','date','Query','About','sentiment'],encoding='latin-1'); #skiping these two rows as they have some bad data
testData=pd.read_csv('drive/MyDrive/data_set/sts-test_data_set/testdata.csv', names=['lebal','id','date','Query','About','sentiment'],encoding='latin-1'); #test_data
print(tweetsDatas.head())
print(tweetsDatas.lebal.value_counts())

   lebal          id                          date     Query            About  \
0      0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY  _TheSpecialOne_   
1      0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY    scotthamilton   
2      0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY         mattycus   
3      0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY          ElleCTF   
4      0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY           Karoli   

                                           sentiment  
0  @switchfoot http://twitpic.com/2y1zl - Awww, t...  
1  is upset that he can't update his Facebook by ...  
2  @Kenichan I dived many times for the ball. Man...  
3    my whole body feels itchy and like its on fire   
4  @nationwideclass no, it's not behaving at all....  
0    800000
4    800000
Name: lebal, dtype: int64


In [ ]:
tweetsData=tweetsDatas.sample(n=1000000,replace=True)
tweetsData_train=tweetsData[0:10000]
tweetsData_train.lebal.value_counts()

0    5056
4    4944
Name: lebal, dtype: int64

In [ ]:
df=tweetsData_train
# max_length is used for max sequence of input
max_length = get_max_length(df)

train_X, train_Y = preprocess(df)

# Build Model

In [ ]:
# LSTM model
model = Sequential()
model.add(LSTM(32))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train

In [ ]:
train_Y

array([[1, 0],
       [1, 0],
       [1, 0],
       ...,
       [0, 1],
       [0, 1],
       [1, 0]])

In [ ]:
print('Train...')
model.fit(train_X, train_Y,epochs=50)

Train...
Epoch 1/50
313/313 [==============================] - 8s 19ms/step - loss: 0.6551 - accuracy: 0.6088
Epoch 2/50
313/313 [==============================] - 6s 19ms/step - loss: 0.6093 - accuracy: 0.6724
Epoch 3/50
313/313 [==============================] - 6s 19ms/step - loss: 0.5891 - accuracy: 0.6851
Epoch 4/50
313/313 [==============================] - 6s 19ms/step - loss: 0.5759 - accuracy: 0.7051
Epoch 5/50
313/313 [==============================] - 7s 22ms/step - loss: 0.5683 - accuracy: 0.7050
Epoch 6/50
313/313 [==============================] - 6s 19ms/step - loss: 0.5634 - accuracy: 0.7086
Epoch 7/50
313/313 [==============================] - 6s 19ms/step - loss: 0.5559 - accuracy: 0.7118
Epoch 8/50
313/313 [==============================] - 7s 21ms/step - loss: 0.5466 - accuracy: 0.7173
Epoch 9/50
313/313 [==============================] - 6s 19ms/step - loss: 0.5447 - accuracy: 0.7208
Epoch 10/50
313/313 [==============================] - 6s 19ms/step - loss: 0.5347

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                36224     
                                                                 
 dense (Dense)               (None, 2)                 66        
                                                                 
Total params: 36,290
Trainable params: 36,290
Non-trainable params: 0
_________________________________________________________________


# Test
your model can predict correctly even for unseen words from training.  
This is the most benefit of using pretrained word embedding.  
Why? pretrained embedding will encode [better], [nice] to similar vector of [best]  
even if these words were not in train.  
therefore, the input vector to RNN is similar, so correct answers for even these unseen words.

In [ ]:
test_df=tweetsData[100001:100100]
print(test_df.lebal.value_counts())

test_X, test_Y = preprocess(test_df)

score, acc = model.evaluate(test_X, test_Y, verbose=2)
print('Test score:', score)
print('Test accuracy:', acc)

0    54
4    45
Name: lebal, dtype: int64
4/4 - 0s - loss: 1.2463 - accuracy: 0.6869 - 42ms/epoch - 10ms/step
Test score: 1.246348261833191
Test accuracy: 0.6868686676025391


In [ ]:
test_df=testData
print(test_df.lebal.value_counts())

test_X, test_Y = preprocess(test_df)

score, acc = model.evaluate(test_X, test_Y, verbose=2)
print('Test score:', score)
print('Test accuracy:', acc)